In [1]:
import pandas as pd

regiaon descriptor:

•	Kircher regions not matching any cCRE region for which we have predictions ("not_matching")
                                                
•	The part of the Kircher regions prior to their first cCRE matches ("pre_cre");

•	The part of the Kircher regions after their last cCRE matches ("post_cre");

•	The part of the Kircher regions between two consecutive cCREs that they matche ("inter_cre");

•	The above three combined together ("all_missing") (union of above three).

•   The Kircher regions that is fully included in cCREs ('cres_full.txt')

In [2]:
HEADER = ['Kircher_region', 'length']
DIR = '/hpc/group/igvf/benchmarks/Kircher_et_al_2019'
COORD = '/work/igvf-pm/K562/mdl_eval/BlueSTARR_vs_DeltaSVM/Kircher_region.coords'
OUT = '/work/igvf-pm/K562/mdl_eval/BlueSTARR_vs_DeltaSVM/Kircher_region_chunk.txt'
ALPHA = ['G','C','A','T']

In [3]:
print('# Kircher region: 21 = unmatched + full (without cutting)')
unmatched = pd.read_table(DIR+'/kircher_not_matching.txt', names = HEADER)
print('# unmatched: ', len(unmatched))
pre_cre = pd.read_table(DIR+'/kircher_matching_pre_cre.txt', names = HEADER)
print('# pre cre: ', len(pre_cre))
post_cre = pd.read_table(DIR+'/kircher_matching_post_cre.txt', names = HEADER)
print('# post cre: ', len(post_cre))
inter_cre = pd.read_table(DIR+'/kircher_matching_inter_cre.txt', names = HEADER)
print('# inter cre: ', len(inter_cre))
full_cre = pd.read_table(DIR+'/kircher_matching_cres_full.txt', names = HEADER)
print('# full cre: ', len(full_cre))
all_missing = pd.read_table(DIR+'/kircher_matching_all_missing.txt', names = HEADER)
print('# all missing: ', len(all_missing), '= pre + post + inter')
print('# Kirchir region cut by cCRE:', len(unmatched)+len(all_missing), '= unmatched + all_missing')

# Kircher region: 21 = unmatched + full (without cutting)
# unmatched:  5
# pre cre:  7
# post cre:  8
# inter cre:  11
# full cre:  16
# all missing:  26 = pre + post + inter
# Kirchir region cut by cCRE: 31 = unmatched + all_missing


1. Simple approach: center the 300bp over the region (and mutate only bases within the region), regardless of subset. The four exceptions aside, this would still ensure at least *some* cCRE sequence is included. For the non-matching regions <300bp, center the 300bp window

In [4]:
def get_coordinates(df, label, seqlen=300):
    # old begin/end: coor in Kircher's region
    # global begin/end: actual pred region, also the cood for 2bittofa
    # being/end: region mutate bases
    # local begin/end: local idx for begin/end in region [global_begin, global_end] to get nucleotide
    
    halflen = seqlen/2
    df['chrom'] = df['Kircher_region'].str.split(':').str[0]
    df['old_begin'] = df['Kircher_region'].str.split(':').str[-1].str.split('-').str[0].astype(int)
    df['old_end'] = df['Kircher_region'].str.split(':').str[-1].str.split('-').str[-1].astype(int)
    df['global_begin'] = ((df['old_begin'] + df['old_end'])/2 - halflen).astype(int)
    df['global_end'] = ((df['old_begin'] + df['old_end'])/2 + halflen).astype(int)

    # length > 300, will manually update later
    df.loc[df['old_begin'] <= df['global_begin'], 'begin'] = df['global_begin']    
    df.loc[df['old_begin'] <= df['global_begin'], 'end'] = df['global_end']
    df.loc[df['old_begin'] <= df['global_begin'], 'local_begin'] = 0
    df.loc[df['old_begin'] <= df['global_begin'], 'local_end'] = seqlen

    # length <= 300
    df.loc[df['old_begin'] > df['global_begin'], 'begin'] = df['old_begin']
    df.loc[df['old_begin'] > df['global_begin'], 'end'] = df['old_end']
    df.loc[df['old_begin'] > df['global_begin'], 'local_begin'] = df['old_begin'] - df['global_begin']
    df.loc[df['old_begin'] > df['global_begin'], 'local_end'] = df['old_end'] - df['global_begin']

    df['begin'] = df['begin'].astype(int)
    df['end'] = df['end'].astype(int)        
    df['label'] = [label]*len(df)

    return df

In [5]:
unmatched = get_coordinates(unmatched, 'unmatched')
pre_cre = get_coordinates(pre_cre, 'pre_cre')
post_cre = get_coordinates(post_cre, 'post_cre')
inter_cre = get_coordinates(inter_cre, 'inter_cre')
# all_missing = get_coordinates(all_missing, 'all_missing')
all = pd.concat([unmatched, pre_cre, post_cre, inter_cre], ignore_index=True)
all

,Kircher_region,length,chrom,old_begin,old_end,global_begin,global_end,begin,end,local_begin,local_end,label
0,chr8:127519269-127519732,463,chr8,127519269,127519732,127519350,127519650,127519350,127519650,0.0,300.0,unmatched
1,chr10:112998239-112998839,600,chr10,112998239,112998839,112998389,112998689,112998389,112998689,0.0,300.0,unmatched
2,chrX:139530462-139530765,303,chrX,139530462,139530765,139530463,139530763,139530463,139530763,0.0,300.0,unmatched
3,chr11:5227021-5227208,187,chr11,5227021,5227208,5226964,5227264,5227021,5227208,57.0,244.0,unmatched
4,chr11:5249804-5250078,274,chr11,5249804,5250078,5249791,5250091,5249804,5250078,13.0,287.0,unmatched
5,chr2:60494939-60495301,362,chr2,60494939,60495301,60494970,60495270,60494970,60495270,0.0,300.0,pre_cre
6,chr6:37807498-37807632,134,chr6,37807498,37807632,37807415,37807715,37807498,37807632,83.0,217.0,pre_cre
7,chr7:156791118-156791361,243,chr7,156791118,156791361,156791089,156791389,156791118,156791361,29.0,272.0,pre_cre
8,chr22:19723265-19723293,28,chr22,19723265,19723293,19723129,19723429,19723265,19723293,136.0,164.0,pre_cre
9,chr1:155301394-155301406,12,chr1,155301394,155301406,155301250,155301550,155301394,155301406,144.0,156.0,pre_cre


In [6]:
# correct coordinates for regin > 300bp

# 463 and 303bp not matching any cCRE: Use the first 300bp and the last 300bp to predict the first and second halves, respectively. (The idea is to avoid including sequence context outside of the designated region. )
# in this case, old begin/end = begin/end since we are cutting Kircher regions by mutating regions
# 'local_begin'] = df['old_begin'] - df['global_begin']
# 'local_end'] = df['old_end'] - df['global_begin']

first_half_463 = {'Kircher_region': 'chr8:127519269-127519732', 'length':232, 'chrom':'chr8', 'old_begin': 127519269, 'old_end':127519269+232, 'global_begin':127519269, 'global_end':127519269+300,
                  'begin':127519269, 'end':127519269+232, 'local_begin':0, 'local_end':232, 'label':'unmatched'} 
second_half_463 = {'Kircher_region': 'chr8:127519269-127519732', 'length':231, 'chrom':'chr8', 'old_begin': 127519269+232, 'old_end':127519732, 'global_begin':127519732-300, 'global_end':127519732,
                  'begin':127519269+232, 'end':127519732, 'local_begin':300-231, 'local_end':300, 'label':'unmatched'} 

first_half_303 = {'Kircher_region': 'chrX:139530462-139530765', 'length':152, 'chrom':'chrX', 'old_begin': 139530462, 'old_end':139530462+152, 'global_begin':139530462, 'global_end':139530462+300,
                  'begin':139530462, 'end':139530462+152, 'local_begin':0, 'local_end':152, 'label':'unmatched'} 
second_half_303 = {'Kircher_region': 'chrX:139530462-139530765', 'length':151, 'chrom':'chrX', 'old_begin': 139530462+152, 'old_end':139530765, 'global_begin':139530765-300, 'global_end':139530765,
                  'begin':139530462+152, 'end':139530765, 'local_begin':300-151, 'local_end':300, 'label':'unmatched'} 

# 362bp preceding a matching cCRE (“pre_cre”): Use the first 300bp to predict the first half (181bp) then treat the second half as above.
first_half_362 = {'Kircher_region': 'chr2:60494939-60495301', 'length':181, 'chrom':'chr2', 'old_begin': 60494939, 'old_end':60494939+181, 'global_begin':60494939, 'global_end':60494939+300,
                  'begin':60494939, 'end':60494939+181, 'local_begin':0, 'local_end':181, 'label':'pre_cre'} 
second_half_362 = {'Kircher_region': 'chr2:60494939-60495301', 'length':181, 'chrom':'chr2', 'old_begin': 60494939+181, 'old_end':60495301, 'global_begin':60495301-300, 'global_end':60495301,
                  'begin':60494939+181, 'end':60495301, 'local_begin':300-181, 'local_end':300, 'label':'pre_cre'} 

# 600bp not matching any cCRE: Use thirds: 0-300 to predict first 200 (= 0-200), 150-450 to predict second 200 (= 200-400), 300-600 to predict last 200 (400-600).
first_600 = {'Kircher_region': 'chr10:112998239-112998839', 'length':200, 'chrom':'chr10', 'old_begin': 112998239, 'old_end':112998239+200, 'global_begin':112998239, 'global_end':112998239+300,
                  'begin':112998239, 'end':112998239+200, 'local_begin':0, 'local_end':200, 'label':'unmatched'} 
second_600 = {'Kircher_region': 'chr10:112998239-112998839', 'length':200, 'chrom':'chr10', 'old_begin': 112998239+200, 'old_end':112998239+400, 'global_begin':112998239+150, 'global_end':112998239+450,
                  'begin':112998239+200, 'end':112998239+400, 'local_begin':50, 'local_end':250, 'label':'unmatched'} 
third_600 = {'Kircher_region': 'chr10:112998239-112998839', 'length':200, 'chrom':'chr10', 'old_begin': 112998239+400, 'old_end':112998839, 'global_begin':112998239+300, 'global_end':112998839,
                  'begin':112998239+400, 'end':112998839, 'local_begin':100, 'local_end':300, 'label':'unmatched'} 

df_exceptions = pd.concat([pd.DataFrame([first_half_463]), pd.DataFrame([second_half_463]), pd.DataFrame([first_half_303]), pd.DataFrame([second_half_303]), pd.DataFrame([first_half_362]), pd.DataFrame([second_half_362]),
                          pd.DataFrame([first_600]), pd.DataFrame([second_600]), pd.DataFrame([third_600])], ignore_index=True)
df_exceptions

,Kircher_region,length,chrom,old_begin,old_end,global_begin,global_end,begin,end,local_begin,local_end,label
0,chr8:127519269-127519732,232,chr8,127519269,127519501,127519269,127519569,127519269,127519501,0,232,unmatched
1,chr8:127519269-127519732,231,chr8,127519501,127519732,127519432,127519732,127519501,127519732,69,300,unmatched
2,chrX:139530462-139530765,152,chrX,139530462,139530614,139530462,139530762,139530462,139530614,0,152,unmatched
3,chrX:139530462-139530765,151,chrX,139530614,139530765,139530465,139530765,139530614,139530765,149,300,unmatched
4,chr2:60494939-60495301,181,chr2,60494939,60495120,60494939,60495239,60494939,60495120,0,181,pre_cre
5,chr2:60494939-60495301,181,chr2,60495120,60495301,60495001,60495301,60495120,60495301,119,300,pre_cre
6,chr10:112998239-112998839,200,chr10,112998239,112998439,112998239,112998539,112998239,112998439,0,200,unmatched
7,chr10:112998239-112998839,200,chr10,112998439,112998639,112998389,112998689,112998439,112998639,50,250,unmatched
8,chr10:112998239-112998839,200,chr10,112998639,112998839,112998539,112998839,112998639,112998839,100,300,unmatched


In [7]:
all = pd.concat([all, df_exceptions], ignore_index=True)
drop_idx = all[(all['Kircher_region'] == 'chr8:127519269-127519732') & (all['length'] == 463)].index.to_list() + all[(all['Kircher_region'] == 'chrX:139530462-139530765') & (all['length'] == 303)].index.to_list() + \
            all[(all['Kircher_region'] == 'chr2:60494939-60495301') & (all['length'] == 362)].index.to_list() + all[(all['Kircher_region'] == 'chr10:112998239-112998839') & (all['length'] == 600)].index.to_list()
all = all.drop(index=drop_idx)
all = all.reset_index(drop=True)
all

,Kircher_region,length,chrom,old_begin,old_end,global_begin,global_end,begin,end,local_begin,local_end,label
0,chr11:5227021-5227208,187,chr11,5227021,5227208,5226964,5227264,5227021,5227208,57.0,244.0,unmatched
1,chr11:5249804-5250078,274,chr11,5249804,5250078,5249791,5250091,5249804,5250078,13.0,287.0,unmatched
2,chr6:37807498-37807632,134,chr6,37807498,37807632,37807415,37807715,37807498,37807632,83.0,217.0,pre_cre
3,chr7:156791118-156791361,243,chr7,156791118,156791361,156791089,156791389,156791118,156791361,29.0,272.0,pre_cre
4,chr22:19723265-19723293,28,chr22,19723265,19723293,19723129,19723429,19723265,19723293,136.0,164.0,pre_cre
5,chr1:155301394-155301406,12,chr1,155301394,155301406,155301250,155301550,155301394,155301406,144.0,156.0,pre_cre
6,chr10:46046243-46046273,30,chr10,46046243,46046273,46046108,46046408,46046243,46046273,135.0,165.0,pre_cre
7,chr19:11089230-11089248,18,chr19,11089230,11089248,11089089,11089389,11089230,11089248,141.0,159.0,pre_cre
8,chr6:396487-396593,106,chr6,396487,396593,396390,396690,396487,396593,97.0,203.0,post_cre
9,chr1:209816238-209816390,152,chr1,209816238,209816390,209816164,209816464,209816238,209816390,74.0,226.0,post_cre


In [8]:
# prepare cood file to get fasta seq via 2bittofa using global begin/end
coordFile = open(COORD,"wt")
for i in range(len(all)):
    print(all.loc[i, 'chrom']+":"+str(all.loc[i, 'global_begin'])+'-'+str(all.loc[i, 'global_end']),file=coordFile)
coordFile.close()

2bittifa command:

twoBitToFa -noMask -seqList=/work/igvf-pm/K562/mdl_eval/BlueSTARR_vs_DeltaSVM/Kircher_region.coords /work/igvf-pm/A549/extra_GCs/hg38.2bit /work/igvf-pm/K562/mdl_eval/BlueSTARR_vs_DeltaSVM/Kircher_region.fasta

In [9]:
def loadFasta(fasta_path, as_dict=False,uppercase=False, stop_at=None,
              revcomp=False):
    fastas = []
    seq = None
    header = None
    for r in (gzip.open(fasta_path) if fasta_path.endswith(".gz") else open(fasta_path)):
        if type(r) is bytes: r = r.decode("utf-8")
        r = r.strip()
        if r.startswith(">"):
            if seq != None and header != None:
                fastas.append([header, seq])
                if stop_at != None and len(fastas) >= stop_at:
                    break
            seq = ""
            header = r[1:]
        else:
            if seq != None:
                seq += r.upper() if uppercase else r
            else:
                seq = r.upper() if uppercase else r
    if stop_at != None and len(fastas) < stop_at:
        fastas.append([header, seq])
    elif stop_at == None:
        fastas.append([header, seq])
    if as_dict:
        return {h: s for h, s in fastas}
    if(revcomp):
        for rec in fastas:
            rc=generate_complementary_sequence(rec[1])
            rec[1]=rec[1]+"NNNNNNNNNNNNNNNNNNNN"+rc
    return pd.DataFrame({'location': [e[0] for e in fastas],
                         'sequence': [e[1] for e in fastas]})

In [10]:
fasta = loadFasta('/work/igvf-pm/K562/mdl_eval/BlueSTARR_vs_DeltaSVM/Kircher_region.fasta')

In [23]:
# make prediction chunks
outFile = open(OUT,"wt")
count = 0
for i in range(len(all)):
    localEnd = int(all.loc[i, 'local_end'])
    localBegin = int(all.loc[i, 'local_begin'])
    begin = all.loc[i, 'begin']
    end = all.loc[i, 'end']
    seq = fasta.loc[i, 'sequence']
    head = all.loc[i, 'chrom']+":"+str(all.loc[i, 'old_begin'])+"-"+str(all.loc[i, 'old_end'])+'\t'

    for i in range(0,localEnd-localBegin):
        head+=str(begin+i)+':ref='+seq[localBegin+i]+':'+','.join([x for x in ALPHA])+'\t'
        count += 1
    print(head,file=outFile)

In [24]:
# sanity check
all['length'].sum() == count

True

In [25]:
count

4567

In [28]:
count*3

13701

In [ ]:
# deletions?? 21 regions delte every single base? entire k's dataset?

In [37]:
import math
pred = pd.read_table('/work/igvf-pm/K562/mdl_eval/BlueSTARR_vs_DeltaSVM/Kircher_region_pred.txt', names=['Kircher_region','actual_interval','pos','ref','alt','pred'])

spdi = {'chr1':'NC_000001.11', 'chr2':'NC_000002.12', 'chr3':'NC_000003.12', 'chr4':'NC_000004.12', 'chr5':'NC_000005.10', 'chr6':'NC_000006.12',
        'chr7':'NC_000007.14', 'chr8':'NC_000008.11', 'chr9':'NC_000009.12', 'chr10':'NC_000010.11', 'chr11':'NC_000011.10', 'chr12':'NC_000012.12',
        'chr13':'NC_000013.11', 'chr14':'NC_000014.9', 'chr15':'NC_000015.10', 'chr16':'NC_000016.10', 'chr17':'NC_000017.11', 'chr18':'NC_000018.10',
        'chr19':'NC_000019.10', 'chr20':'NC_000020.11', 'chr21':'NC_000021.9', 'chr22':'NC_000022.11', 'chrX':'NC_000023.11', 'chrY':'NC_000024.10'}

pred['pred'] = pred['pred']*math.log2(math.e)  # convert prediction to log2 scale
pred['ref'] = pred['ref'].apply(lambda x: x[4:])

ref = pred.loc[pred['ref'] == pred['alt'], ]
ref = ref.drop(columns=['alt'])
ref = ref.rename(columns={'pred':'ref_pred'})
merged = pd.merge(pred, ref, on=['Kircher_region','actual_interval','pos','ref'], how='left')
merged['effect'] = merged['pred'] - merged['ref_pred']
merged = merged.loc[merged['ref'] != merged['alt'], ]
merged['chrom'] = merged['Kircher_region'].apply(lambda x: x.split(':')[0])
merged['Kircher_start'] = merged['Kircher_region'].apply(lambda x: (x.split(':')[-1]).split('-')[0])
merged['Kircher_end'] = merged['Kircher_region'].apply(lambda x: (x.split(':')[-1]).split('-')[-1])
merged['pos'] = merged['pos'].apply(lambda x: x[4:])
merged['SPDI'] = merged['chrom'].map(spdi)+':'+merged['pos']+':'+merged['ref']+':'+merged['alt']
merged = merged[['chrom','Kircher_start','Kircher_end','effect','SPDI']]
merged

,chrom,Kircher_start,Kircher_end,effect,SPDI
1,chr11,5227021,5227208,-0.022929,NC_000011.10:5227021:G:C
2,chr11,5227021,5227208,-0.036303,NC_000011.10:5227021:G:A
3,chr11,5227021,5227208,-0.026123,NC_000011.10:5227021:G:T
5,chr11,5227021,5227208,-0.023237,NC_000011.10:5227022:G:C
6,chr11,5227021,5227208,-0.010791,NC_000011.10:5227022:G:A
...,...,...,...,...,...
18261,chr10,112998639,112998839,0.009463,NC_000010.11:112998837:A:C
18263,chr10,112998639,112998839,-0.004954,NC_000010.11:112998837:A:T
18264,chr10,112998639,112998839,0.020276,NC_000010.11:112998838:T:G
18265,chr10,112998639,112998839,0.020303,NC_000010.11:112998838:T:C


In [38]:
merged.to_csv('/work/igvf-pm/K562/mdl_eval/BlueSTARR_vs_DeltaSVM/Kircher_pred_processed.txt', index=False, sep='\t')